In [ ]:
import pandas as pd

In [ ]:
import os
os.environ['PATH'] = "/usr/local/sbin:/usr/local/bin:/usr/sbin:/opt/R/4.2.3/bin:/usr/bin:/sbin:/bin:/opt/miniconda3/conda/bin:/opt/miniconda3/bin"
%load_ext rpy2.ipython

In [ ]:
#! git clone https://github.com/slagtermaarten/genesets.git ~/libs/genesets/
#! git clone https://github.com/slagtermaarten/maartenutils.git ~/libs/maartenutils/

In [ ]:
#%%R
#if (!require(genesets)) {
#  devtools::install('~/libs/genesets')
#}

# if (!require(genesets)) {
#   devtools::install('~/libs/maartenutils')
# }

In [ ]:
%%R
library(magrittr)
library("data.table")
library(edgeR)
library(flexgsea)
library(foreach)
library(doParallel)

In [ ]:
%%R
devtools::load_all('~/libs/genesets')
source("https://raw.githubusercontent.com/slagtermaarten/TONIC/refs/heads/master/R/GSEA_funcs_paired.R")

In [ ]:
%%R
devtools::load_all('~/libs/maartenutils')

In [ ]:
%%R
source("https://raw.githubusercontent.com/slagtermaarten/TONIC/refs/heads/master/R/misc.R")
patient_labels <- readRDS('../../processed_data/patient_labels.rds')

In [ ]:
%%R
mut_load_var_types <- 
 c('Missense Variant', 'Frameshift Variant', 'Stop Gained', 
   'Conservative Inframe Insertion', 'Conservative Inframe Deletion', 
   'Disruptive Inframe Insertion', 'Disruptive Inframe Deletion',
   'Structural Interaction Variant', 
   # 'TF Binding Site Variant', 
   'Stop Lost', 
   # 'Start Lost', 
   'Protein Protein Contact', 'Stop Retained Variant') %>% sort
# tolower(paste(mut_load_var_types, collapse = ', '))

timepoints <- c('baseline' = 'Baseline', 
#'post.induction' = 'Post-induction', 
                'on.nivo' = 'On nivo')
it_timepoints <- timepoints
timepoints_inv <- setNames(names(timepoints), timepoints)

## Labels for timepoints 1, 2 and 3
timepoint_labels <- c('baseline', 
#                      'post_induction', 
                      'on_nivo')


treatment_arms <- c('Radiotherapy', 'Doxorubicin', 'Cyclophosphamide',
                    'Cisplatin', 'No induction')
## Ordering by clinical benefit and NanoString results
treatment_arms <- c('No induction', 'Radiotherapy', 'Cyclophosphamide',
                    'Cisplatin', 'Doxorubicin')
label_reps <- c('efron_thisted_estimator' = 'TCRSeq repertoire size', 
                'adaptive_t_cells' = 'TCRSeq %T-cells',
                'sample_clonality' = 'TCRSeq clonality',
                'ca15_3' = 'Baseline CA15.3',
                's_til' = 'Baseline stromal TIL [%]',
                'pd_l1_immunoinfiltrate' = 'Baseline PD-L1 [%]')

In [ ]:
ann = pd.read_csv('../../processed_data/tonic_final_not_full_metadata_response_add_sets.csv', sep=',', index_col=0)
two_tp = [x for x in ann[ann.Timepoint == 'On_nivo'].StudyID.values.tolist() if x in ann[ann.Timepoint == 'Baseline'].StudyID.values.tolist()]
ann_two_tp = ann[ann.StudyID.isin(two_tp)]

In [ ]:
ann_two_tp = ann_two_tp[(ann_two_tp.Cohort != 'T1_1') | (ann_two_tp.Induction != 'Control')]

In [ ]:
rna_sample_annotation = ann_two_tp.reset_index().rename(columns = {'Timepoint': 'timepoint', 'index': 'cf_number', 'StudyID': 'patient', 'Induction': 'arm', 'Response': 'clinical_response'})
rna_sample_annotation = rna_sample_annotation.loc[:, ['patient', 'cf_number', 'timepoint', 'arm', 'clinical_response']].replace({'timepoint': {'Postinduction': 'Post-induction', 'On_nivo': 'On nivo'}, 
                                                                                                                                'arm': {'Control': 'No induction'}})
rna_sample_annotation = rna_sample_annotation[rna_sample_annotation.timepoint != 'Post-induction']
rna_sample_annotation = rna_sample_annotation[rna_sample_annotation.arm.isin(['No induction', 'Doxorubicin', 'Cisplatin'])]
rna_sample_annotation = rna_sample_annotation[rna_sample_annotation.clinical_response == 'R']

In [ ]:
rna_sample_annotation.arm.value_counts()

In [ ]:
rna_sample_annotation.arm = 'Combined'

In [ ]:
%R -i rna_sample_annotation

In [ ]:
%%R
rna_sample_annotation <- as.data.table(rna_sample_annotation, keep.rownames = T)
rna_sample_annotation$timepoint <- factor(rna_sample_annotation$timepoint)
rna_sample_annotation$arm <- factor(rna_sample_annotation$arm)
local_run <- F

In [ ]:
%%R
library(readxl)
library(naturalsort)
data_dir = './'


rna_sample_annotation[, timepoint := factor(timepoints[as.integer(timepoint)], levels = timepoints)]

setkey(rna_sample_annotation, patient)

rna_sample_annotation <- rna_sample_annotation[naturalsort::naturalorder(patient)]
rna_sample_annotation[, cf_number := tolower(cf_number)]

In [ ]:
tmm = pd.read_csv('../../processed_data/TMM_counts_all_TONIC_batch_corrected.csv',
                         index_col=0)
%R -i tmm

In [ ]:
%%R
tmm <- as.data.table(tmm, keep.rownames = T)
rownames(tmm) <- tmm$rn
tmm[ , rn:=NULL]
colnames(tmm) <- tolower(colnames(tmm))

In [ ]:
%%R
my_paired_WC_test <- function(x, y, abs = F) {
  cur_pats <- unique(y$patient)
  pre_idx <- vapply(cur_pats,
                    function(pat) which(y$patient == pat &
                                        y$timepoint == 'Baseline'),
                    integer(1))
  post_idx <- vapply(cur_pats,
                     function(pat) which(y$patient == pat &
                                         y$timepoint == 'On nivo'),
                     integer(1))
  ret_val <- apply(x, 2, function(r) {
    w_test <- suppressWarnings(
              tryCatch(wilcox.test(x = r[pre_idx], y = r[post_idx],
                                   paired = T),
                       error = function(e) { print(e); browser() }))
    t_val <- (1 - w_test$p.value) *
      (log2(median(r[post_idx]) + 1) - log2(median(r[pre_idx]) + 1))
    return(t_val)
  })
  ret_val[is.na(ret_val)] <- 0
  # rownames(rna_read_counts_salmon_tmm_M)
  # ret_val[which(is.nan(ret_val))[1]]
  ret_val <- as.matrix(ret_val)
  colnames(ret_val) <- 't_val'
  if (abs) {
    ret_val <- abs(ret_val)
  }
  return(ret_val)
}

In [ ]:
%%R
gsea_all_arms <- function(
  gene_sets = HALLMARK_pathways,
  patients = rna_sample_annotation[!is.na(clinical_response), unique(patient)],
  allowed_timepoints = c('Baseline', 'On nivo'),
  gene_score_fn = my_unpaired_WC_test,
  exp_mat = rna_read_counts_salmon_tmm_M,
  resp_exp = 'timepoint',
  paired_test = F,
  nperm = 1000,
  abs = F,
  fn_extra = '') {

  if (!exists('rna_sample_annotation')) source('R/load_rna_dat.R')

  arm_specific_gsea <-
    lapply(rna_sample_annotation[, levels(arm)], 
      function(l_arm) {
      l_patients <- intersect(rna_sample_annotation[, patient], patients)
      if (l_arm != 'All arms') {
        l_patients <- intersect(l_patients,
          rna_sample_annotation[arm %in% l_arm, patient])
      }
      res <- gsea_wrapper(gene_sets = gene_sets,
        patients = l_patients,
        allowed_timepoints = allowed_timepoints,
        gene_score_fn = gene_score_fn,
        exp_mat = exp_mat,
        resp_exp = resp_exp,
        paired_test = paired_test,
        nperm = nperm,
        abs = abs,
        fn_extra = fn_extra)
      if (!is.null(res)) 
        res <- cbind(res$gsea, 'arm' = l_arm)
      return(res)
    }) %>% rbindlist(fill = T)

  if (!null_dat(arm_specific_gsea)) {
    arm_specific_gsea$arm <- factor(arm_specific_gsea$arm,
      levels = treatment_arms)
  }

  return(arm_specific_gsea)
}

In [ ]:
%%R
registerDoParallel(cores=20)
res_new <- gsea_wrapper(gene_sets = HALLMARK_pathways,
  patients = rna_sample_annotation[arm == 'Combined', unique(patient)],
  gene_score_fn = my_paired_WC_test,
  paired_test = T,
allowed_timepoints =  c('Baseline', 'On nivo'),
  resp_exp = 'timepoint',
  exp_mat = tmm,
  nperm = 1000,
  abs = F,
  fn_extra = '')

In [ ]:
%R -o res_new

In [ ]:
res_new['gsea'].to_csv('../../processed_data/flexgsea_hallmarks_on_nivo_combined_responders.tsv', sep='\t')